<a href="https://colab.research.google.com/github/kenneith/myGit/blob/master/tecka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance ta openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2a5eb1ae6aa749c75ce570eace8abe782884d457ceb3e9b161b2c4c5ae9b6af1
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
from google.colab import files

uploaded = files.upload()  # 上傳 stock-input.xlsx 和 tecka.txt（或其他名字）

Saving stock-input.xlsx to stock-input.xlsx


In [ ]:
import os
os.listdir(".")  # 確認 stock-input.xlsx 存在

['.config', 'stock-input.xlsx', 'sample_data']

In [ ]:
# -*- coding: utf-8 -*-
"""
tecka.py - 台股月線技術分析系統（依資料長度分級處理）
功能：
- 資料 <24個月：僅顯示收盤價與月份數，其他指標為「不適用」
- 24~60個月：除 MACD 外，其他指標正常計算
- >=60個月：全部指標計算
- 匯出至 tecka-output.xlsx
"""

import yfinance as yf
import pandas as pd

# 導入 ta 套件
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator, StochasticOscillator, WilliamsRIndicator
from ta.volatility import BollingerBands
from ta.volume import OnBalanceVolumeIndicator


# -----------------------------
# 設定檔案路徑
# -----------------------------
input_file = 'stock-input.xlsx'
output_file = 'tecka-output.xlsx'


# -----------------------------
# Step 1: 讀取第一個且唯一的 Sheet
# -----------------------------
try:
    excel_file = pd.ExcelFile(input_file)
    sheet_name = excel_file.sheet_names[0]
    print(f"📁 檢測到工作表：{sheet_name}")

    stock_df = pd.read_excel(input_file, sheet_name=sheet_name)
    stock_df.columns = stock_df.columns.str.strip()

    required_cols = ['上市櫃', '股票代號', '股票簡稱']
    missing_cols = [col for col in required_cols if col not in stock_df.columns]
    if missing_cols:
        raise ValueError(f"❌ 缺少必要欄位：{missing_cols}")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ 找不到 '{input_file}'")
except Exception as e:
    raise Exception(f"❌ 讀取 Excel 錯誤：{str(e)}")


# -----------------------------
# 初始化結果清單
# -----------------------------
results = []


# -----------------------------
# 迴圈處理每一檔股票
# -----------------------------
for _, row in stock_df.iterrows():
    try:
        market_str = str(row['上市櫃']).strip()
        code = str(row['股票代號']).strip()
        name = str(row['股票簡稱']).strip()

        # 判斷後綴
        if market_str == '上市':
            suffix = '.TW'
        elif market_str == '上櫃':
            suffix = '.TWO'
        else:
            print(f"⚠️ 無效市場別跳過：{code}({name}) -> '{market_str}'")
            continue

        ticker = f"{code}{suffix}"
        print(f"🔍 分析中：{ticker} ({name})")

        # 下載月線資料
        data = yf.download(ticker, interval='1mo', period='5y', auto_adjust=False)
        if data.empty:
            print(f"⚠️ 資料為空：{ticker}")
            result = {
                '市場別': market_str,
                '股票代號': code,
                '股票簡稱': name,
                '最新收盤': '不適用',
                '月資料長度': 0,
            }
            indicators = ['MA', 'KD', 'MACD', 'RSI', 'PSY', 'BIAS', 'W%R', '布林通道', 'OBV']
            for ind in indicators:
                result[f'{ind}_數值'] = '不適用'
                result[f'{ind}_分析'] = '不適用'
            results.append(result)
            continue

        # 確保為一維
        df_raw = pd.DataFrame({
            'Open':   data['Open'].squeeze(),
            'High':   data['High'].squeeze(),
            'Low':    data['Low'].squeeze(),
            'Close':  data['Close'].squeeze(),
            'Volume': data['Volume'].squeeze(),
        }).dropna()

        n_months_raw = len(df_raw)

        # 取得最新收盤價（永遠保留）
        latest_close = df_raw['Close'].iloc[-1] if len(df_raw) > 0 else '不適用'

        # ✅ 情況 1：原始資料 < 24 個月 → 僅保留收盤價與月數
        if n_months_raw < 24:
            print(f"🟡 數據長度不足，只有 {n_months_raw} 個月：{code}")
            result = {
                '市場別': market_str,
                '股票代號': code,
                '股票簡稱': name,
                '最新收盤': round(latest_close, 2),
                '月資料長度': n_months_raw,
            }
            indicators = ['MA', 'KD', 'MACD', 'RSI', 'PSY', 'BIAS', 'W%R', '布林通道', 'OBV']
            for ind in indicators:
                result[f'{ind}_數值'] = '不適用'
                result[f'{ind}_分析'] = '不適用'
            results.append(result)
            continue

        # -----------------------------
        # 資料 >=24 個月 → 開始計算指標
        # -----------------------------
        df = df_raw.copy()

        # MA (5,10,20)
        for w in [5, 10, 20]:
            df[f'MA_{w}'] = df['Close'].rolling(w).mean()

        # KD
        stoch = StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3)
        df['%K'] = stoch.stoch()
        df['%D'] = stoch.stoch_signal()

        # MACD（需要至少 35 個月才可靠）
        macd_val, macd_signal = None, None
        if n_months_raw >= 35:
            macd = MACD(close=df['Close'], window_fast=12, window_slow=26, window_sign=9)
            df['MACD'] = macd.macd()
            df['MACD_signal'] = macd.macd_signal()
            macd_val = df['MACD'].iloc[-1]
            macd_signal = df['MACD_signal'].iloc[-1]

        # RSI
        rsi = RSIIndicator(close=df['Close'], window=14)
        df['RSI'] = rsi.rsi()

        # W%R
        wr = WilliamsRIndicator(high=df['High'], low=df['Low'], close=df['Close'], lbp=14)
        df['W%R'] = wr.williams_r()

        # 布林通道
        bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
        df['BB_upper'] = bb.bollinger_hband()
        df['BB_lower'] = bb.bollinger_lband()
        df['BB_middle'] = bb.bollinger_mavg()

        # OBV
        obv = OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume'])
        df['OBV'] = obv.on_balance_volume()

        # PSY (12個月)
        df['PSY_12'] = (df['Close'].diff() > 0).rolling(12).mean() * 100

        # BIAS (10個月)
        df['MA_10'] = df['Close'].rolling(10).mean()
        df['BIAS_10'] = (df['Close'] - df['MA_10']) / df['MA_10'] * 100

        # 再次 dropna
        df_clean = df.dropna()
        if len(df_clean) < 1:
            print(f"⚠️ 計算後無有效數據，只有 {n_months_raw} 個月：{code}")
            result = {
                '市場別': market_str,
                '股票代號': code,
                '股票簡稱': name,
                '最新收盤': round(latest_close, 2),
                '月資料長度': n_months_raw,
            }
            indicators = ['MA', 'KD', 'MACD', 'RSI', 'PSY', 'BIAS', 'W%R', '布林通道', 'OBV']
            for ind in indicators:
                result[f'{ind}_數值'] = '不適用'
                result[f'{ind}_分析'] = '不適用'
            results.append(result)
            continue

        # 取最後一筆有效資料
        latest = df_clean.iloc[-1]
        prev = df_clean.iloc[-2] if len(df_clean) >= 2 else df_clean.iloc[-1]

        # 實際取得月份數（上限60）
        n_months = min(n_months_raw, 60)

        # -----------------------------
        # 提取各指標數值
        # -----------------------------
        ma5, ma10, ma20 = latest.get('MA_5'), latest.get('MA_10'), latest.get('MA_20')
        k_val, d_val = latest.get('%K'), latest.get('%D')
        rsi_val = latest.get('RSI')
        wr_val = latest.get('W%R')
        bb_upper, bb_middle, bb_lower = latest.get('BB_upper'), latest.get('BB_middle'), latest.get('BB_lower')
        obv_val = latest.get('OBV')
        psy_val = latest.get('PSY_12')
        bias_val = latest.get('BIAS_10')

        # -----------------------------
        # 各指標趨勢判斷（偏多 / 偏空 / 中性）
        # -----------------------------

        def safe_trend(cond, fallback):
            return "偏多" if cond else ("偏空" if fallback else "中性")

        result = {
            '市場別': market_str,
            '股票代號': code,
            '股票簡稱': name,
            '最新收盤': round(latest_close, 2),
            '月資料長度': n_months,

            # MA
            'MA_數值': f"MA5={ma5:.2f}, MA10={ma10:.2f}, MA20={ma20:.2f}" if all(pd.notna([ma5, ma10, ma20])) else '不適用',
            'MA_分析': safe_trend(
                cond=(ma5 > ma10 > ma20 and ma5 > prev['MA_5'] and ma10 > prev['MA_10']),
                fallback=(ma5 < ma10 < ma20 and ma5 < prev['MA_5'] and ma10 < prev['MA_10'])
            ) if all(pd.notna([ma5, ma10, ma20])) else '不適用',

            # KD
            'KD_數值': f"%K={k_val:.2f}, %D={d_val:.2f}" if pd.notna(k_val) and pd.notna(d_val) else '不適用',
            'KD_分析': safe_trend(
                cond=(k_val > d_val and k_val > 50),
                fallback=(k_val < d_val and k_val < 50)
            ) if pd.notna(k_val) and pd.notna(d_val) else '不適用',

            # MACD（僅當 n_months_raw >= 35 時才計算）
            'MACD_數值': f"DIF={macd_val:.2f}, DEM={macd_signal:.2f}" if n_months_raw >= 35 else '不適用',
            'MACD_分析': safe_trend(
                cond=(macd_val > macd_signal and macd_val > 0),
                fallback=(macd_val < macd_signal and macd_val < 0)
            ) if n_months_raw >= 35 else '不適用',

            # RSI
            'RSI_數值': f"{rsi_val:.2f}" if pd.notna(rsi_val) else '不適用',
            'RSI_分析': safe_trend(
                cond=(55 <= rsi_val <= 70),
                fallback=(30 <= rsi_val <= 45)
            ) if pd.notna(rsi_val) else '不適用',

            # PSY
            'PSY_數值': f"{psy_val:.2f}%" if pd.notna(psy_val) else '不適用',
            'PSY_分析': safe_trend(
                cond=(55 <= psy_val <= 75),
                fallback=(25 <= psy_val <= 45)
            ) if pd.notna(psy_val) else '不適用',

            # BIAS
            'BIAS_數值': f"{bias_val:+.2f}%" if pd.notna(bias_val) else '不適用',
            'BIAS_分析': safe_trend(
                cond=(3 <= bias_val <= 6),
                fallback=(-6 <= bias_val <= -3)
            ) if pd.notna(bias_val) else '不適用',

            # W%R
            'W%R_數值': f"{wr_val:.2f}" if pd.notna(wr_val) else '不適用',
            'W%R_分析': safe_trend(
                cond=(-80 <= wr_val <= -100),
                fallback=(-0 >= wr_val > -20)
            ) if pd.notna(wr_val) else '不適用',

            # 布林通道
            '布林通道_數值': f"上:{bb_upper:.2f}, 中:{bb_middle:.2f}, 下:{bb_lower:.2f}" if all(pd.notna([bb_upper, bb_middle, bb_lower])) else '不適用',
            '布林通道_分析': safe_trend(
                cond=(latest['Close'] > bb_middle and latest['Close'] < bb_upper * 0.95),
                fallback=(latest['Close'] < bb_middle and latest['Close'] > bb_lower * 1.05)
            ) if all(pd.notna([bb_upper, bb_middle, bb_lower])) else '不適用',

            # OBV
            'OBV_數值': f"{obv_val:,.0f}" if pd.notna(obv_val) else '不適用',
            'OBV_分析': (
                "偏多" if len(df['OBV'].dropna()) >= 3 and df['OBV'].dropna().tail(3).iloc[-1] > df['OBV'].dropna().tail(3).iloc[-2] > df['OBV'].dropna().tail(3).iloc[-3] else
                "偏空" if len(df['OBV'].dropna()) >= 3 and df['OBV'].dropna().tail(3).iloc[-1] < df['OBV'].dropna().tail(3).iloc[-2] < df['OBV'].dropna().tail(3).iloc[-3] else
                "中性"
            ) if pd.notna(obv_val) else '不適用',
        }

        results.append(result)

    except Exception as e:
        print(f"❌ 分析失敗 {code}({name}): {str(e)}")
        result = {
            '市場別': market_str,
            '股票代號': code,
            '股票簡稱': name,
            '最新收盤': '無法取得' if 'latest_close' not in locals() else round(latest_close, 2),
            '月資料長度': n_months_raw if 'n_months_raw' in locals() else 0,
        }
        for ind in ['MA', 'KD', 'MACD', 'RSI', 'PSY', 'BIAS', 'W%R', '布林通道', 'OBV']:
            result[f'{ind}_數值'] = '不適用'
            result[f'{ind}_分析'] = '不適用'
        results.append(result)


# -----------------------------
# Step 2: 匯出結果至 Excel
# -----------------------------
if results:
    output_df = pd.DataFrame(results)

    # 計算偏多/偏空計數（跳過「不適用」）
    analysis_cols = [
        'MA_分析', 'KD_分析', 'MACD_分析',
        'RSI_分析', 'PSY_分析', 'BIAS_分析',
        'W%R_分析', '布林通道_分析', 'OBV_分析'
    ]
    output_df['偏多計數'] = output_df[analysis_cols].apply(
        lambda x: (x == "偏多").sum() if any(v != "不適用" for v in x) else "不適用", axis=1)
    output_df['偏空計數'] = output_df[analysis_cols].apply(
        lambda x: (x == "偏空").sum() if any(v != "不適用" for v in x) else "不適用", axis=1)

    def overall_trend(row):
        if row['偏多計數'] == "不適用":
            return "不適用"
        try:
            pos = int(row['偏多計數'])
            neg = int(row['偏空計數'])
            if pos >= 6:
                return "強勢偏多"
            elif neg >= 6:
                return "明顯偏空"
            elif pos > neg:
                return "整體偏多"
            elif neg > pos:
                return "整體偏空"
            else:
                return "多空平衡"
        except:
            return "不適用"

    output_df['綜合判斷'] = output_df.apply(overall_trend, axis=1)

    # 調整欄位順序
    cols = [
        '市場別', '股票代號', '股票簡稱', '最新收盤', '月資料長度',
        '偏多計數', '偏空計數', '綜合判斷',
        'MA_數值', 'MA_分析',
        'KD_數值', 'KD_分析',
        'MACD_數值', 'MACD_分析',
        'RSI_數值', 'RSI_分析',
        'PSY_數值', 'PSY_分析',
        'BIAS_數值', 'BIAS_分析',
        'W%R_數值', 'W%R_分析',
        '布林通道_數值', '布林通道_分析',
        'OBV_數值', 'OBV_分析'
    ]
    output_df = output_df[cols]

    # 匯出 Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        output_df.to_excel(writer, index=False, sheet_name='技術分析報告')

    print(f"\n✅ 成功分析 {len(results)} 檔股票，結果已匯出至 '{output_file}'")
else:
    print("⚠️ 無任何有效分析結果")

📁 檢測到工作表：Sheet1
🔍 分析中：2476.TW (鉅祥)


[*********************100%***********************]  1 of 1 completed


🔍 分析中：3622.TW (洋華)


[*********************100%***********************]  1 of 1 completed


🔍 分析中：5471.TW (松翰)


...

🔍 分析中：4916.TW (事欣科)


[*********************100%***********************]  1 of 1 completed


🔍 分析中：3073.TWO (天方能源)


[*********************100%***********************]  1 of 1 completed


✅ 成功分析 35 檔股票，結果已匯出至 'tecka-output.xlsx'
